In [1]:
from scholars import *
X = np.genfromtxt('data/X_train.txt', delimiter=',')
Y = np.genfromtxt('data/Y_train.txt', delimiter=',')
X_test = np.genfromtxt('data/X_test.txt', delimiter=',')
X, Y = ml.shuffleData(X, Y)
Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.75) 
np.random.seed(0)

In [2]:
def stackPredictions(predictions):
    num_data = len(predictions[0])
    finalPrediction = [0]*num_data
    for j in range(num_data):
        finalPrediction[j] = np.mean(np.array(predictions)[:, j])
    return finalPrediction


def convertToFinalPredictions(pred):
    final_pred = []
    for i in pred:
        if i>0.5 or i == True:
            final_pred.append(1)
        else:
            final_pred.append(0)

    return final_pred

In [3]:
#TRAINS LEARNER
gradientBoostTR = GradientBoost(Xtr,Ytr)
logistic41featuresTR = LogisticRegression().fit(Xtr[:,:41],Ytr)
adaBoosTR = AdaBoost(Xtr,Ytr, learning_rate=2)
gradientBoost2TR = GradientBoost2(Xtr,Ytr)
randomForestTR = RandomForest(Xtr,Ytr,nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=25)
randomForest2TR = RandomForest2(Xtr,Ytr)
knn28categoricalTR = ml.knn.knnClassify(Xtr[:,41:69],Ytr)
gradient12Boost2TR = GradientBoost2(Xtr[:,:69], Ytr)
gradient23Boost2TR = GradientBoost2(Xtr[:,41:], Ytr)
newXtr = np.hstack((Xtr[:,:41],Xtr[:,69:]))    
gradient13Boost2TR = GradientBoost2(newXtr, Ytr)

gradientBoost = GradientBoost(X,Y)
logistic41features = LogisticRegression().fit(X[:,:41],Y)
knn28categorical = ml.knn.knnClassify(X[:,41:69],Y)
adaBoost = AdaBoost(X,Y, learning_rate=2)
gradientBoost2 = GradientBoost2(X,Y)
randomForest = RandomForest(X,Y,nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=25)
randomForest2 = RandomForest2(X,Y)
gradient12Boost2 = GradientBoost2(X[:,:69], Y)
gradient23Boost2 = GradientBoost2(X[:,41:], Y)
newX = np.hstack((X[:,:41],X[:,69:]))    
gradient13Boost2 = GradientBoost2(newX, Y)


In [180]:
#RUNS PREDICTION
gradientPredictionTR = gradientBoostTR.predict(Xva)[:, 1]
adaPredictionTR = adaBoosTR.predict(Xva)[:, 1]
gradientPrediction2TR = gradientBoost2TR.predict(Xva)
forestPredictionTR = randomForestTR.predict(Xva)
forest2PredictionTR = randomForest2TR.predict(Xva)
logistic41PredictionTR = logistic41featuresTR.predict(Xva[:,:41])
knn28PredictionTR = knn28categoricalTR.predict(Xva[:,41:69])
newXva = np.hstack((Xva[:,:41],Xva[:,69:]))
gradient13PredictionTR = gradient13Boost2TR.predict(newXva)  
gradient23PredictionTR = gradient23Boost2TR.predict(Xva[:,41:])
gradient12PredictionTR = gradient12Boost2TR.predict(Xva[:,:69])

gradientPrediction = gradientBoost.predict(X_test)[:, 1]
adaPrediction = adaBoost.predict(X_test)[:, 1]
gradientPrediction2 = gradientBoost2.predict(X_test)
forestPrediction = randomForest.predict(X_test)
forest2Prediction = randomForest2.predict(X_test)
logistic41Prediction = logistic41features.predict(X_test[:,:41])
knn28Prediction = knn28categorical.predict(X_test[:,41:69])
newXtest = np.hstack((X_test[:,:41],X_test[:,69:]))
gradient13Prediction = gradient13Boost2.predict(newXtest)  
gradient23Prediction = gradient23Boost2.predict(X_test[:,41:])
gradient12Prediction = gradient12Boost2.predict(X_test[:,:69])


In [189]:

forest2PredictionTR = convertToFinalPredictions(forest2PredictionTR)
print("forest2Prediction ERROR: ",1-sum(forest2PredictionTR==Yva)/float(len(Yva)))

forestPredictionTR = convertToFinalPredictions(forestPredictionTR)
print("forestPrediction ERROR: ",1-sum(forestPredictionTR==Yva)/float(len(Yva)))

gradientPrediction2TR = convertToFinalPredictions(gradientPrediction2TR)
print("gradientPrediction2 ERROR: ",1-sum(gradientPrediction2TR==Yva)/float(len(Yva)))

gradientPredictionTR = convertToFinalPredictions(gradientPredictionTR)
print("gradientPrediction ERROR: ",1-sum(gradientPredictionTR==Yva)/float(len(Yva)))

adaPredictionTR = convertToFinalPredictions(adaPredictionTR)
print("adaPrediction ERROR: ",1-sum(adaPredictionTR==Yva)/float(len(Yva)))

logistic41PredictionTR = convertToFinalPredictions(logistic41PredictionTR)
print("LOGISTIC ERROR: ", 1-sum(logistic41PredictionTR==Yva)/float(len(Yva)))

knn28PredictionTR = convertToFinalPredictions(knn28PredictionTR)
print("knn28PredictionTR ERROR: ", 1-sum(knn28PredictionTR==Yva)/float(len(Yva)))


gradient13PredictionTR = convertToFinalPredictions(gradient13PredictionTR)
print("gradient13PredictionTR ERROR: ",1-sum(gradient13PredictionTR==Yva)/float(len(Yva)))

gradient12PredictionTR = convertToFinalPredictions(gradient12PredictionTR)
print("gradient12PredictionTR ERROR: ",1-sum(gradient12PredictionTR==Yva)/float(len(Yva)))

gradient23PredictionTR = convertToFinalPredictions(gradient23PredictionTR)
print("gradient23PredictionTR ERROR: ",1-sum(gradient23PredictionTR==Yva)/float(len(Yva)))


forest2Prediction ERROR:  0.3394396551724138
forestPrediction ERROR:  0.3556034482758621
gradientPrediction2 ERROR:  0.3351293103448276
gradientPrediction ERROR:  0.34213362068965514
adaPrediction ERROR:  0.42403017241379315
LOGISTIC ERROR:  0.4385775862068966
knn28PredictionTR ERROR:  0.4951508620689655
gradient13PredictionTR ERROR:  0.33728448275862066
gradient12PredictionTR ERROR:  0.33836206896551724
gradient23PredictionTR ERROR:  0.4132543103448276


In [236]:

jBlend = [gradientBoost2TR,randomForest2TR,gradient13Boost2TR,gradientBoostTR, gradient12Boost2TR,adaBoosTR]
jBlendPred = np.array([gradientPrediction2TR,forest2PredictionTR,gradient13PredictionTR,gradientPredictionTR,forestPredictionTR,adaPredictionTR])
def calculateWeight(predictions,Yva):
    # allagreeCounter = 0
    # allagreeButWrong = 0
    # mostAgree = 0
    # mostAgreeButWrong = 0
    # equalProb = 0
    # adaWrong = 0
    numOfleaners = len(predictions)
    weight = np.zeros(numOfleaners)
    for indx, trueClass in enumerate(Yva):
        avgPredict = np.mean(predictions[:,indx])
        # if avgPredict == 1 or avgPredict == 0: # if all the learners agree or disagree we can't change the weight in any ways
        #     pass
        # if avgPredict != 0.5: # optimized heavily because there is a majority here
        #     if avgPredict == 5/6:
        #         for i in range(numOfleaners):
        #             if predictions[i,indx] == trueClass:
        #                 weight[i] += 1
        #             else:
        #                 weight[i] -= 5
        #     elif avgPredict == 1/6:
        #         for i in range(numOfleaners):
        #             if predictions[i,indx] == trueClass:
        #                 weight[i] += 500
        #             else:
        #                 weight[i] -= 1
        #     elif avgPredict == 2/6:
        #         for i in range(numOfleaners):
        #             if predictions[i,indx] == trueClass:
        #                 weight[i] += 200
        #             else:
        #                 weight[i] -= 1
        #     elif avgPredict == 4/6:
        #         for i in range(numOfleaners):
        #             if predictions[i,indx] == trueClass:
        #                 weight[i] += 1
        #             else:
        #                 weight[i] -= 2
        # else: # can be optimized slightly. half  of the learners agree and half disagree
            # for i in range(numOfleaners):
            #     if predictions[i,indx] == trueClass:
            #         weight[i] += 1
            #     else:
            #         weight[i] -= 1
        for i in range(numOfleaners):
            if predictions[i,indx] == trueClass:
                weight[i] += 1
            else:
                weight[i] -= 1
            
    #     if avgPredict == 1 or avgPredict == 0:
    #         allagreeCounter +=1
    #         if avgPredict != trueClass:
    #             allagreeButWrong +=1
    #     elif avgPredict == 5/6 or avgPredict == 1/6 or avgPredict == 2/6 or avgPredict == 4/6:
    #         mostAgree+=1
    #         if avgPredict > 0.5 and trueClass == 0 or avgPredict < 0.5 and trueClass ==1:
    #             mostAgreeButWrong +=1
    #     else:
    #         equalProb += 1
    

    # print(f"Percent that all agree: {allagreeCounter/len(Yva)}, all agree but wrong: {allagreeButWrong/len(Yva)}")
    # print(f"Percent that can be heavily optimize: {mostAgree/len(Yva)}, most agree but wrong: {mostAgreeButWrong/len(Yva)}")
    # print(f"Percent that can be slightly optimize: {equalProb/len(Yva)}")
    # print(len(Yva))
    # print(weight)
    return weight/sum(weight)
    #     weight = np.array([1/len(predictions)]*len(predictions))
weight = calculateWeight(jBlendPred,Yva)
print(weight)

[0.19029851 0.18532338 0.18781095 0.18221393 0.16666667 0.08768657]


In [237]:
def computeBlendErr(pred,weight):
    result = 0
    for i in range(len(pred)):
        result += np.array(pred[i]) * weight[i]
    printFinalVaErr(result)

In [238]:

computeBlendErr(jBlendPred,weight)

0.3275862068965517


In [202]:
jBlendPrediction = 0.19343618   *np.array(gradientPrediction2) +0.19222626  * np.array(forest2Prediction) + 0.18436177 * np.array(gradient13Prediction)+ 0.18935269 *np.array(gradientPrediction)+0.17876588 *np.array(forestPrediction)+0.06185723 *np.array(adaPrediction)

Y_test = np.vstack((np.arange(X_test.shape[0]), jBlendPrediction)).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('JBlendPredictions.txt', Y_test, '%d, %.2f',header='ID,Predicted', delimiter=',')

In [204]:
#finalPrediction = stackPredictions([adaPrediction, forestPrediction, gradientPrediction, forest2Prediction, gradientPrediction2])

#BEST
#finalPrediction = 0.15*np.array(adaPrediction)+0.1*np.array(forestPrediction)+ 0.3*np.array(forest2Prediction) + 0.15*np.array(gradientPrediction) + 0.3*np.array(gradientPrediction2)
#BEST

#finalPrediction = 0.3*np.array(adaPrediction)+0*np.array(forestPrediction)+ 0.1*np.array(forest2Prediction) + 0.1*np.array(gradientPrediction) + 0.5*np.array(gradientPrediction2)
#finalPrediction = 0.1*np.array(adaPrediction) + 0.05*np.array(forestPrediction)+ 0.2*np.array(forest2Prediction) + 0.2*np.array(gradientPrediction) + 0.35*np.array(gradientPrediction2) + 0.1*np.array(logistic41Prediction)

#finalPrediction = 0.1*np.array(adaPrediction) + 0.05*np.array(forestPrediction)+ 0.2*np.array(forest2Prediction) + 0.2*np.array(gradientPrediction) + 0.42*np.array(gradientPrediction2) + 0.02*np.array(logistic41Prediction) + 0.01*np.array(knn28Prediction)

finalPredictionBESTtr = 0.15*np.array(adaPredictionTR)+0.1*np.array(forestPredictionTR)+ 0.3*np.array(forest2PredictionTR) + 0.15*np.array(gradientPredictionTR) + 0.3*np.array(gradientPrediction2TR)
finalPredictionTR = 0.15*np.array(adaPredictionTR)+0.1*np.array(forestPredictionTR)+ 0.3*np.array(forest2PredictionTR) + 0.2*np.array(gradientPredictionTR) + 0.25*np.array(gradientPrediction2TR)
finalPrediction = 0.05*np.array(adaPrediction)+0.1*np.array(forestPrediction)+ 0.15*np.array(forest2Prediction) + 0.15*np.array(gradientPrediction) + 0.25*np.array(gradientPrediction2) + 0.125*np.array(gradient13Prediction) + 0.125*np.array(gradient12Prediction)+ 0.05*np.array(gradient23Prediction)


In [205]:
def printFinalVaErr(finalPrediction):
    errCount = 0
    ndata=len(finalPrediction)
    for i in range(ndata):
        if(not ((finalPrediction[i]<0.5 and Yva[i]<0.5) or (finalPrediction[i]>0.5 and Yva[i]>0.5))):
            errCount += 1
    print(errCount/float(ndata))

In [206]:
printFinalVaErr(finalPredictionBESTtr)

0.3394396551724138


In [28]:
printFinalVaErr(finalPredictionTR)

0.33459051724137934


In [29]:
Y_test = np.vstack((np.arange(X_test.shape[0]), finalPrediction)).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('FinalPredictions.txt', Y_test, '%d, %.2f',header='ID,Predicted', delimiter=',')

In [30]:
#stacking outputs
outputs = [forest2PredictionTR, forestPredictionTR, gradientPrediction2TR, gradientPredictionTR, adaPredictionTR, logistic41PredictionTR, knn28PredictionTR, gradient13PredictionTR, gradient12PredictionTR, gradient23PredictionTR]
#outputs = [forest2PredictionTR, forestPredictionTR, gradientPrediction2TR, gradientPredictionTR, adaPredictionTR]

new_outputs = []
for i in outputs:
    new_outputs.append([[j] for j in i])
xMetaTR = np.hstack((np.array(new_outputs)))


outputs = [forest2Prediction, forestPrediction, gradientPrediction2, gradientPrediction, adaPrediction, logistic41Prediction, knn28Prediction, gradient13Prediction, gradient12Prediction, gradient23Prediction]
new_outputs = []
for i in outputs:
    new_outputs.append([[j] for j in i])
xMeta = np.hstack((np.array(new_outputs)))




In [31]:
stacked_gradient2 = GradientBoost2(xMetaTR, Yva)
stacked_gradient = GradientBoost(xMetaTR, Yva)
stacked_ada = AdaBoost(xMetaTR, Yva)


In [32]:
stacked_forest2 = RandomForest2(xMetaTR, Yva)
stacked_forest = RandomForest(xMetaTR, Yva, nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=50)

In [41]:
stacked_gradient_predict = stacked_gradient.predict(xMeta)[:,1]
stacked_gradient2_predict = stacked_gradient2.predict(xMeta)
stacked_ada_predict = stacked_ada.predict(xMeta)[:, 1]
stacked_forest2_predict = stacked_forest2.predict(xMeta)
stacked_forest_predict = stacked_forest.predict(xMeta)


In [34]:
#finalPrediction = 0.15*np.array(stacked_ada_predict)+0.1*np.array(stacked_forest_predict)+ 0.3*np.array(stacked_forest2_predict) + 0.15*np.array(stacked_gradient_predict) + 0.3*np.array(stacked_gradient2_predict)

#finalPrediction = 0.25*np.array(stacked_forest_predict)+ 0.25*np.array(stacked_forest2_predict) + 0.25*np.array(stacked_gradient_predict) + 0.25*np.array(stacked_gradient2_predict)




In [42]:
final1 = 0.15*np.array(stacked_ada_predict)+0.1*np.array(stacked_forest_predict)+ 0.3*np.array(stacked_forest2_predict) + 0.15*np.array(stacked_gradient_predict) + 0.3*np.array(stacked_gradient2_predict)
final2 = 0.15*np.array(adaPrediction)+0.1*np.array(forestPrediction)+ 0.3*np.array(forest2Prediction) + 0.15*np.array(gradientPrediction) + 0.3*np.array(gradientPrediction2)

In [43]:
finalPrediction = 0.4*final1 + 0.6*final2

In [44]:
print(finalPrediction)
print(final2)

[0.36234344 0.21815255 0.53636606 ... 0.4712536  0.8746315  0.44363959]
[0.3880978  0.23610827 0.52470947 ... 0.46591101 0.8607841  0.47953562]


In [48]:
print(finalPrediction)

[0.36234344 0.21815255 0.53636606 ... 0.4712536  0.8746315  0.44363959]
